# OB Files for 2024-10-29 to 2024-11-01 observing session
Priorities for this session:
* Continue to double the exposure time returned by the PEPSI ETC in hopes of getting good SNR
* Prioritize systems that weren't resolved with DSSI measurements from APO
* Get multiple RV measurements on a systems for which we already have LBT spectra


In [1]:
from functools import partial
from sqlite3 import connect
import sys

import astroplan as ap
from astropy.table import Table
from astropy.time import Time
import astropy.units as u
import pandas as pd

sys.path.append("../../")
import astropaul.targetlistcreator as tlc
import astropaul.lbt as lbt


In [23]:
# start by making list of targets from 2024B proposal that are observable and have prior spectra
conn = connect("../../django/TargetDB/db.sqlite3")

observer = ap.Observer.at_site("LBT")
session = (Time("2024-10-29"), Time("2024-11-02"))

creator = tlc.TargetListCreator(connection=conn)
creator.steps = [
    tlc.add_targets,
    tlc.add_tess,
    tlc.add_coords,
    partial(tlc.hide_cols, prefix="tess"),
    tlc.add_lists,
    tlc.add_spectra,
    partial(lbt.add_pepsi_params, fiber="300", cd_blue=3, cd_red=6, snr=100, binocular=True),
    partial(tlc.add_observability, observer=observer, time_segment=session),
]
targets = creator.calculate()
# targets = Table.from_pandas(creator.calculate())

criteria = (
    (targets["list_LBT_2024B"])
    & (targets["pepsi_count"] > 0)
    & (targets["observable"])
)
lbt_targets = targets.loc[criteria]

# double the exposure time and convert from seconds to minutes
lbt_targets.loc[:, "pepsi_exp_time"] *= 2
lbt_targets.loc[:, "pepsi_exp_time"] /= 60

# prioritize targets unresolved by APO speckle
lbt_targets.loc[:, "pepsi_priority"] = "Normal"
lbt_targets.loc[lbt_targets["list_HQND"], "pepsi_priority"] = "High"

Table.from_pandas(lbt_targets)


c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\erfa\core.py:4620: RuntimeWarning: invalid value encountered in ld
  p1 = ufunc.ld(bm, p, q, e, em, dlim)
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\erfa\core.py:19004: RuntimeWarning: invalid value encountered in anp
  c_retval = ufunc.anp(a)
c:\Users\User\AppData\Local\Programs\Python\Python312\Lib\site-packages\erfa\core.py:4036: RuntimeWarning: invalid value encountered in atioq
  aob, zob, hob, dob, rob = ufunc.atioq(ri, di, astrom)


target_name,target_source,target_type,tess_version,tess_HIP,tess_TYC,tess_UCAC,tess_TWOMASS,tess_SDSS,tess_ALLWISE,tess_GAIA,tess_APASS,tess_KIC,tess_objType,tess_typeSrc,tess_ra,tess_dec,tess_POSflag,tess_pmRA,tess_e_pmRA,tess_pmDEC,tess_e_pmDEC,tess_PMflag,tess_plx,tess_e_plx,tess_PARflag,tess_gallong,tess_gallat,tess_eclong,tess_eclat,tess_Bmag,tess_e_Bmag,tess_Vmag,tess_e_Vmag,tess_umag,tess_e_umag,tess_gmag,tess_e_gmag,tess_rmag,tess_e_rmag,tess_imag,tess_e_imag,tess_zmag,tess_e_zmag,tess_Jmag,tess_e_Jmag,tess_Hmag,tess_e_Hmag,tess_Kmag,tess_e_Kmag,tess_TWOMflag,tess_prox,tess_w1mag,tess_e_w1mag,tess_w2mag,tess_e_w2mag,tess_w3mag,tess_e_w3mag,tess_w4mag,tess_e_w4mag,tess_GAIAmag,tess_e_GAIAmag,tess_Tmag,tess_e_Tmag,tess_TESSflag,tess_SPFlag,tess_Teff,tess_e_Teff,tess_logg,tess_e_logg,tess_MH,tess_e_MH,tess_rad,tess_e_rad,tess_mass,tess_e_mass,tess_rho,tess_e_rho,tess_lumclass,tess_lum,tess_e_lum,tess_d,tess_e_d,tess_ebv,tess_e_ebv,tess_numcont,tess_contratio,tess_disposition,tess_duplicate_id,tess_priority,tess_eneg_EBV,tess_epos_EBV,tess_EBVflag,tess_eneg_Mass,tess_epos_Mass,tess_eneg_Rad,tess_epos_Rad,tess_eneg_rho,tess_epos_rho,tess_eneg_logg,tess_epos_logg,tess_eneg_lum,tess_epos_lum,tess_eneg_dist,tess_epos_dist,tess_distflag,tess_eneg_Teff,tess_epos_Teff,tess_TeffFlag,tess_gaiabp,tess_e_gaiabp,tess_gaiarp,tess_e_gaiarp,tess_gaiaqflag,tess_starchareFlag,tess_VmagFlag,tess_BmagFlag,tess_splists,tess_e_RA,tess_e_Dec,tess_RA_orig,tess_Dec_orig,tess_e_RA_orig,tess_e_Dec_orig,tess_raddflag,tess_wdflag,tess_objID,tess_Identifier,ra,dec,pmra,pmdec,parallax,Vmag,Teff,list_Featured_targets,list_Gemini_2024B_Proposal,list_Gemini_2025A_Proposal,list_HQND,list_Kostov_2022,list_Kostov_2023,list_LBT_2023-12-06,list_LBT_2023B,list_LBT_2024B,list_Speckle_Resolved,pepsi_count,pepsi_fiber,pepsi_cd_blue,pepsi_cd_blue_num_exp,pepsi_cd_red,pepsi_cd_red_num_exp,pepsi_snr,pepsi_exp_time,pepsi_priority,pepsi_notes,observable
str13,str28,str6,float64,object,object,str10,str16,object,str19,str19,str8,object,str4,str7,float64,float64,str7,float64,float64,float64,float64,object,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str19,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,str5,str5,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,float64,object,float64,float64,float64,float64,float64,object,str8,str8,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,str9,float64,float64,float64,float64,float64,float64,float64,bool,bool,bool,bool,bool,bool,bool,bool,bool,bool,int64,str3,int64,int64,int64,int64,int64,float64,str6,str1,bool
TIC 161043618,Kostov 2022 arXiv:2202.05790,QuadEB,20190415.0,--,3861-01090-1,714-052661,14534203+5242569,1237659148849119256,J145342.04+524256.9,1594082407606370176,55620165,--,STAR,tmgaia2,223.425163143389,52.7158479482434,tmgaia2,2.17801,1.0398,-3.59429,1.46315,tgas,2.02761,0.603419,tgas,89.7053761311409,55.7930770142449,188.47564384935,63.5887549816856,12.397,0.298,12.497,0.033,14.6131,0.00506893,12.7875,0.00145113,12.3934,0.00145221,12.2906,0.00198318,12.8784,0.00464124,11.329,0.024,11.081,0.017,11.042,0.021,AAA-222-111-000-0-0,--,11.006,0.023,11.036,0.021,11.015,0.087,9.264,--,12.2734,0.001479,11.9099,0.0062,gbprp,splin,5860.0,--,--,--,--,--,1.16588,--,--,--,--,--,DWARF,--,--,443.336,--,--,--,39.0,0.000527256867,--,--,0.00074595405,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,tgas,--,--,dered,12.5342,0.00445,11.8457,0.003765,0.0,--,tycho2v3,tycho2b3,--,17.9879268710121,22.6794687070632,223.4251786238,

In [25]:
import importlib
importlib.reload(lbt)

final_targets = Table.from_pandas(lbt_targets)
final_targets["target_name"].info.format = "<14s"
final_targets["Teff"].info.format = ".0f"
final_targets["pepsi_exp_time"].info.format = ".2f"

foo = lbt.write_lbt_ob_files("UVa_Multistar", final_targets)
print(foo)

Project ID: UV-2024B-007
     Title: Characterizing Quadruple and Sextuple Eclipsing Binaries with PEPSI

Instrument: PEPSI PFU

PI: Steven Majewski

Run Contact:
   Name: Steven Majewski (UVA) / Robert Wilson (NASA)
   Phone: srm4n@virginia.edu   / 210.487.0187
   Contact Times: all / all

Conditions Required:
   Moon: Any
   Desired Seeing: Any
   Cloud Cover: Any

Program Summary:

Targets are TESS identified eclipsing quadruple systems (2 sets of Eclipsing Binaries, each showing both a Primary and Secondary Eclipse).

There are three objectives for this run:
1) Better understand the relation between exposure times from the PEPSI exposure time calculator vs the quality of spectra obtained.
2) Accumulate spectra towards the goal of using radial velocity measurements to characterize the orbital dynamics of the systems.

 Target Name        RA         DEC       Vmag  Teff Fiber BLUE Cross Disperser BLUE CD NExp BLUE CD Exp Time RED Cross Disperser RED CD NExp RED CD Exp Time Desired BL